<h1><center>IAGA Summer School 2019</center></h1>

<h1><center>Spherical Harmonic Models 2</center></h1>

In [ ]:
# Import notebook dependencies

import sys
sys.path.append('..')
import matplotlib.pyplot as plt
plt.style.use('seaborn-white')
import numpy as np
import pandas as pd 
from src import sha_lib as sha
import os

# 1. A more tangible example of modelling using spherical harmonics

First, there's a data file with some spatial data to plot.

In [ ]:
# Load the data
lsfile = '../data/external/land_5deg.csv'
lsdata = np.genfromtxt(lsfile, delimiter=',')

x = lsdata[:,0].reshape(36, 72)
y = lsdata[:,1].reshape(36,72)
z = lsdata[:,2].reshape(36,72)

# Plot them
plt.rcParams['figure.figsize'] = [15, 8]
plt.contourf(x,y,z)
plt.colorbar()
plt.show()

The data file consists of binary values (1=land, 0=water) on a 5-degree grid. We now take these data as the input to a global spherical harmonic analysis and calculate a spherical harmonic model with a user specified resolution.

### >> USER INPUT HERE: Set the maximum spherical harmonic degree of the analysis

In [ ]:
nmax = 5 # Max degree

In [ ]:
# These functions are used for the spherical harmonic analysis and synthesis steps
# Firstly compute a spherical harmonic model of degree and order nmax using the input data as plotted above
def sh_analysis(lsdata, nmax):
    npar = (nmax+1)*(nmax+1)
    ndat = len(lsdata)
    lhs  = np.zeros(npar*ndat).reshape(ndat,npar)

    rhs  = np.zeros(ndat)
    line = np.zeros(npar)
    ic   = -1
    for i in range(ndat):
        th  = 90 - lsdata[i][1]
        ph  = lsdata[i][0] 
        rhs[i] = lsdata[i][2]
        cmphi, smphi = sha.csmphi(nmax,ph)
        pnm = sha.pnm_calc(nmax, th)
        for n in range(nmax+1):
            igx = sha.gnmindex(n,0)
            ipx = sha.pnmindex(n,0)
            line[igx] = pnm[ipx]
            for m in range(1,n+1):
                igx = sha.gnmindex(n,m)
                ihx = sha.hnmindex(n,m)
                ipx = sha.pnmindex(n,m)
                line[igx] = pnm[ipx]*cmphi[m]
                line[ihx] = pnm[ipx]*smphi[m]
        lhs[i,:] = line

    shmod  = np.linalg.lstsq(lhs, rhs.reshape(len(lsdata),1), rcond=None)
    return(shmod)


# Now use the model to synthesise values on a 5 degree grid in latitude and longitude
def sh_synthesis(shcofs, nmax):
    newdata =np.zeros(72*36*3).reshape(2592,3)
    ic = 0
    for ilat in range(36):
        delta = 5*ilat+2.5
        lat = 90 - delta
        for iclt in range(72):
            corr  = 5*iclt+2.5
            long  = -180+corr
            colat = 90-lat
            cmphi, smphi = sha.csmphi(nmax,long)
            vals  = np.dot(sha.gh_phi(shcofs, nmax, cmphi, smphi), sha.pnm_calc(nmax, colat))
            newdata[ic,0]=long
            newdata[ic,1]=lat
            newdata[ic,2]=vals
            ic += 1
    return(newdata)

In [ ]:
# Obtain the spherical harmonic coefficients
shmod = sh_analysis(lsdata=lsdata, nmax=nmax)

# Read the model coefficients
shcofs = shmod[0]

# Synthesise the model coefficients on a 5 degree grid
newdata = sh_synthesis(shcofs=shcofs, nmax=nmax)
# Reshape for plotting purposes
x = newdata[:,0].reshape(36, 72)
y = newdata[:,1].reshape(36,72)
z = newdata[:,2].reshape(36,72)

# Plot the results
plt.rcParams['figure.figsize'] = [15, 8]
levels = [-1.5, -0.75, 0, 0.25, 0.5, 0.75, 2.]
plt.contourf(x,y,z)
plt.colorbar()
plt.show()

# 2. Spherical harmonic models with data gaps

What happens if the data set is incomplete? In this section, you can experiment by removing data within a great circle of a specified radius and position. The functions below are used to create the data gap, and the modelling uses functions in the above section again.

In [ ]:
def greatcircle(th1, ph1, th2, ph2):
    th1 = np.deg2rad(th1)
    th2 = np.deg2rad(th2)
    dph = np.deg2rad(dlong(ph1,ph2))

  # Apply the cosine rule of spherical trigonometry
    dth = np.arccos(np.cos(th1)*np.cos(th2) + \
              np.sin(th1)*np.sin(th2)*np.cos(dph))
    return(dth)

def dlong (ph1, ph2):
    ph1 = np.sign(ph1)*abs(ph1)%360   # These lines return a number in the 
    ph2 = np.sign(ph2)*abs(ph2)%360   # range -360 to 360
    if(ph1 < 0): ph1 = ph1 + 360      # Put the results in the range 0-360
    if(ph2 < 0): ph2 = ph2 + 360
    dph = max(ph1,ph2) - min(ph1,ph2) # So the answer is positive and in the
                                      # range 0-360
    if(dph > 180): dph = 360-dph      # So the 'short route' is returned
    return(dph)

def gh_phi(gh, nmax, cp, sp):
    rx = np.zeros(nmax*(nmax+3)//2+1)
    igx=-1
    igh=-1
    for i in range(nmax+1):
        igx += 1
        igh += 1
        rx[igx]= gh[igh]
        for j in range(1,i+1):
            igh += 2
            igx += 1
            rx[igx] = (gh[igh-1]*cp[j] + gh[igh]*sp[j])
    return(rx)

### >> USER INPUT HERE: Set the location and size of the data gap

Colatitude: degrees

Longitude: degrees

Radius: km

In [ ]:
# Remove a section of data centered on colat0, long0 and radius here
colat0 = 100
long0  = -55
radius  = 500

In [ ]:
lsdata_gap = lsdata.copy()
for row in range(len(lsdata_gap)):
    colat = 90 - lsdata_gap[row,1]
    long  = lsdata_gap[row,0]
    if greatcircle(colat, long, colat0, long0) < radius/6371.2:
        lsdata_gap[row,2] = np.nan

print('Blanked out: ', np.count_nonzero(np.isnan(lsdata_gap)))

x_gap = lsdata_gap[:,0].reshape(36, 72)
y_gap = lsdata_gap[:,1].reshape(36,72)
z_gap = lsdata_gap[:,2].reshape(36,72)

# Plot the map with omitted data
plt.contourf(x_gap, y_gap, z_gap)

### >> USER INPUT HERE: Set the maximum spherical harmonic degree of the analysis

In [ ]:
nmax = 5 #Max degree

In [ ]:
# Select the non-nan data
lsdata_gap = lsdata_gap[~np.isnan(lsdata_gap[:,2])]

# Obtain the spherical harmonic coefficients for the incomplete data set
shmod = sh_analysis(lsdata=lsdata_gap, nmax=nmax)

# Read the model coefficients
shcofs = shmod[0]

# Synthesise the model coefficients on a 5 degree grid
newdata = sh_synthesis(shcofs=shcofs, nmax=nmax)
# Reshape for plotting purposes
x_new = newdata[:,0].reshape(36, 72)
y_new = newdata[:,1].reshape(36,72)
z_new = newdata[:,2].reshape(36,72)

Print the maximum and minimum of the synthesised data. How do they compare to the original data, which was composed of only ones and zeroes? How do the max/min change as you vary the data gap size and the analysis resolution? Try this with a large gap, e.g. 5000 km.

In [ ]:
print(np.min(z_new))
print(np.max(z_new))

In [ ]:
# Plot the results with colour scale according to the data
plt.rcParams['figure.figsize'] = [15, 8]
plt.contourf(x_new, y_new, z_new)
plt.colorbar()
plt.show()

Now see what happens when we restrict the colour scale to values between 0 and 1 regardless of the data values (so that both ends of the colour scale are saturated).

In [ ]:
# Plot the results
plt.rcParams['figure.figsize'] = [15, 8]
levels = [0, 0.25, 0.5, 0.75, 1.]
plt.contourf(x_new, y_new, z_new, levels, extend='both')
plt.colorbar()
plt.show()

# 3. Making a spherical harmonic model of the geomagnetic field

There is a file containing virtual observatory (VO) data calculated from the Swarm 3-satellite constellation in this repository. VOs use a local method to provide point estimates of the magnetic field at a specified time and fixed location at satellite altitude. This technique has various benefits, including: 

1. Easier comparison between satellite (moving instrument) and ground observatory (fixed instrument) data, which is particularly useful when studying time changes of the magnetic field, e.g. geomagnetic jerks.
2. One can compute VOs on a regular spatial grid to mitigate the effects of uneven ground observatory coverage.

A brief summary of the method used to calculate each of these VO data points:

1. Swarm track data over four months are divided into 300 globally distributed equal area bins
2. Gross outliers (deviating over 500nT from the CHAOS-6-x7 internal field model) are removed
3. Only data from magnetically quiet, dark times are kept
4. Estimates of the core and crustal fields from CHAOS-6-x7 are subtracted from each datum to give the residuals
5. A local magnetic potential $V$ is fit to the residuals within the cylinder
6. Values of the residual magnetic field in spherical coordinates ($r$, $\theta$, $\phi$) are computed from the obtained magnetic potential using $B=-\nabla V$ at centre of the bin (at 490km altitude)
7.	An estimate of the modelled core field at the VO calculation point is added back onto the residual field to give the total internal magnetic field value

In this section, we will use VOs as input data for a degree 13 spherical harmonic model of the geomagnetic field at 2015.0 and compare our (satellite data only) model to the latest IGRF (computed using both ground and satellite data) at the same time.

In [ ]:
# Read in the VO data
cfile = '../data/external/SwarmVO_IAGASummerSchool.dat'
cols  = ['Year','Colat','Long','Rad','Br','Bt', 'Bp']
bvals = pd.read_csv(cfile, sep='\s+', skiprows=0, header=None, index_col=0,
                    na_values=[99999.00,99999.00000], names=cols, comment='%')
bvals = bvals.dropna()
bvals['Bt']= -bvals['Bt']
bvals['Br']= -bvals['Br']
colnames=['Colat','Long','Rad','Bt','Bp', 'Br']
bvals=bvals.reindex(columns=colnames)
bvals.columns = ['Colat','Long','Rad','X','Y','Z']

# Set the date to 2015.0 (this is the only common date between the VO data file and the IGRF coefficients file)
epoch = 2015.0
# Set the model resolution to 13 (the same as IGRF)
nmax = 13

In [ ]:
def geomagnetic_model(epoch, nmax, data, RREF=6371.2):
    # Select the data for that date
    colat =  np.array(data.loc[epoch]['Colat'])
    nx   = len(colat) # Number of data triples
    ndat = 3*nx    # Total number of data
    elong =  np.array(data.loc[epoch]['Long'])
    rad   =  np.array(data.loc[epoch]['Rad'])
    rhs   =  data.loc[epoch][['X', 'Y', 'Z']].values.reshape(ndat,1)
    npar = (nmax+1)*(nmax+1) # Number of model parameters
    nx   = len(colat) # Number of data triples
    ndat = 3*nx    # Total number of data

    # Arrays for the equations of condition
    lhs  = np.zeros(npar*ndat).reshape(ndat,npar)
    linex = np.zeros(npar); liney = np.zeros(npar); linez = np.zeros(npar)

    iln = 0 # Row counter
    for i in range(nx): # Loop over data triplets (X, Y, Z)
        r  = rad[i]
        th = colat[i]; ph  = elong[i] 
        rpow = sha.rad_powers(nmax, RREF, r)
        cmphi, smphi = sha.csmphi(nmax, ph)
        pnm, xnm, ynm, znm = sha.pxyznm_calc(nmax, th)

        for n in range(nmax+1): # m=0
            igx = sha.gnmindex(n,0) # Index for g(n,0) in gh conventional order
            ipx = sha.pnmindex(n,0) # Index for pnm(n,0) in array pnm
            rfc = rpow[n]
            linex[igx] =  xnm[ipx]*rfc
            liney[igx] =  0
            linez[igx] = znm[ipx]*rfc

            for m in range(1,n+1): # m>0
                igx = sha.gnmindex(n,m) # Index for g(n,m)
                ihx = igx + 1           # Index for h(n,m)
                ipx = sha.pnmindex(n,m) # Index for pnm(n,m)
                cpr = cmphi[m]*rfc
                spr = smphi[m]*rfc
                linex[igx] =  xnm[ipx]*cpr; linex[ihx] =  xnm[ipx]*spr
                liney[igx] =  ynm[ipx]*spr; liney[ihx] = -ynm[ipx]*cpr
                linez[igx] =  znm[ipx]*cpr; linez[ihx] =  znm[ipx]*spr

        lhs[iln,  :] = linex
        lhs[iln+1,:] = liney
        lhs[iln+2 :] = linez
        iln += 3

    shmod = np.linalg.lstsq(lhs, rhs, rcond=None) # Include the monopole
    # shmod = np.linalg.lstsq(lhs[:,1:], rhs, rcond=None) # Exclude the monopole
    gh = shmod[0]
    return(gh)

In [ ]:
coeffs = geomagnetic_model(epoch=epoch, nmax=nmax, data=bvals)

Read in the IGRF coefficients for comparison

In [ ]:
IGRF12_FILE = os.path.abspath('../data/external/igrf12coeffs.txt')
igrf12 = pd.read_csv(IGRF12_FILE, delim_whitespace=True,  header=3)

In [ ]:
igrf12.head()

In [ ]:
# Select the 2015 IGRF coefficients
igrf = np.array(igrf12[str(epoch)])
# Insert a zero in place of the monopole coefficient (this term is NOT included in the IRGF, or any other field model)
igrf = np.insert(igrf, 0, 0)

Compare our model coefficients to IGRF-12

In [ ]:
print("VO only model \t   IGRF12")
for i in range(196):
    print("% 9.1f\t%9.1f" %(coeffs[i], igrf[i]))

### Exercise

The only common date between the VO data file and the IGRF coefficients file is 2015.0. Use VOs to produce a model at a time of your choice between 2014 and 2018 (other than 2015.0 as we did above) and compare your coefficients to the IGRF at that time. **Hints:** For dates other than yyyy.0, you will need to interpolate the VO data. You may also need to interpolate the IGRF coefficients between 2010.0 and 2015.0. The IGRF coefficients between 2015.0 and 2020.0 will need to be extrapolated from the 2015.0 values using the given secular variation predictions.

### Exercise

Create a second spherical harmonic model based on ground observatory data only and compare it to your satellite data only model and to the IGRF. To do this, you can use observatory annual means as the input data, which we have supplied in the file `oamjumpsapplied.dat` in the external data directory. **Hint:** The `mag` module contains a function that will extract all annual mean values for all observatories in a given year. 

### Acknowledgements

The land/sea data file was provided by John Stevenson (BGS).
The VO data were computed by Magnus Hammer (DTU) and the file prepared by Will Brown (BGS).